In [1]:


# pip install pygame==1.9.6
# import cv2
# from google.colab.patches import cv2_imshow
# from google.colab import output
# import os
# os.environ['SDL_VIDEODRIVER']='dummy'

import os
import pygame
import pandas as pd
import random
import time
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import math


# window size
WIDTH = 530
HEIGHT = 360
FPS = 1200 # how fast game is
EPISODENUM = 6000

# colors
WHITE = (255,255,255)
BLACK = (0,0,0)
RED = (255, 0, 0) # RGB
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255,255,0)

#player
class Player(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((25,25))
        self.image.fill(WHITE)
     
        self.rect = self.image.get_rect()
        self.rect.centerx = 15
        self.rect.centery = HEIGHT/2
        
        self.x_speed = 0
        self.y_speed = 0
        
    def update(self,action):
        # Controls
        key_state = pygame.key.get_pressed()
                
        if key_state[pygame.K_w] or action == 0:
            self.y_speed = -20
            
        elif key_state[pygame.K_s] or action == 1:
            self.y_speed = 20
            
        elif key_state[pygame.K_SPACE] or action == 2:
            self.y_speed = 0
            self.x_speed = 0
       
        #Control update 
        self.rect.x += self.x_speed
        self.rect.y += self.y_speed
        
        self.x_speed = 0
        self.y_speed = 0
        
        #Screeen options
        if self.rect.right > WIDTH:
            self.rect.right = WIDTH
            
        if self.rect.left < 0:
            self.rect.left = 0
        
        if self.rect.top < 0:
            self.rect.top = 0
        
        if self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT

    def getCordinates(self):
        return (self.rect.x,self.rect.y)

"""**ENEMY**"""

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((20,20))
        self.image.fill(RED)
      
        self.rect = self.image.get_rect()
        self.y_center = random.randint(10,HEIGHT-10)
        self.x_center = random.randint(WIDTH-265,WIDTH-10)
        self.rect.center = (self.x_center,self.y_center)
        
        self.x_speed = -10
        self.y_speed = 0
             
    def update(self):
        self.rect.x += self.x_speed
        
        if self.rect.left < 0:
            self.y_center = random.randint(10,HEIGHT-10)
            self.x_center = random.randint(WIDTH-265,WIDTH-10)
            self.rect.center = (self.x_center,self.y_center)
            
    def getCordinates(self):
        return (self.rect.x,self.rect.y)

"""**FRIEND**"""

class Friend(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((20,20))
        self.image.fill(YELLOW)
      
        self.rect = self.image.get_rect()
        self.y_center = random.randint(10,HEIGHT-10)
        self.x_center = random.randint(WIDTH-265,WIDTH-10)
        self.rect.center = (self.x_center,self.y_center)
        
        self.x_speed = -10
             
    def update(self):
        self.rect.x += self.x_speed
        
        if self.rect.left < 0:
            self.y_center = random.randint(10,HEIGHT-10)
            self.x_center = random.randint(WIDTH-265,WIDTH-10)
            self.rect.center = (self.x_center,self.y_center)
            
    def getCordinates(self):
        return (self.rect.x,self.rect.y)

class DQLAgent:
    def __init__(self,weights = None):
        # parameter / hyperparameter
        self.state_size = 10   # [distances]
        self.action_size = 3   #  move top, move bottom, stop
        
        self.gamma = 0.99
        self.learning_rate = 0.0001 
        
        # self.epsilon = 1  # explore
        self.epsilon = 0.01  # testing explore
        
        self.epsilon_decay = 0.9993
        self.epsilon_min = 0.01
        
        self.memory = deque(maxlen = 4000) 
        
        self.model = self.build_model(weights)
        self.target_model = self.build_model(weights)
                
    def build_model(self,weights = None):
        # neural network for deep q learning
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu' ))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        
        if weights:
            model.load_weights(weights)
            print("Loaded model from disk")

        return model
    
    def remember(self, state, action, reward, next_state, done):
        # storage
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        state = np.array(state)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        # training
        if (len(self.memory) < batch_size):
            return
        minibatch = random.sample(self.memory,batch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3])) 
            y[not_done_indices] += np.multiply(self.gamma, predict_sprime_target[not_done_indices, np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])
            
        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(batch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=1, verbose=0)
               

    def targetModelUpdate(self):
        self.target_model.set_weights(self.model.get_weights())
 

    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

"""**ENV**"""

class Env:
    def __init__(self,weights = None):
        pygame.sprite.Sprite.__init__(self)
        self.player_group = pygame.sprite.Group()
        self.enemy_group = pygame.sprite.Group()
        self.friend_group = pygame.sprite.Group()
        
        self.player = Player()
        self.player_group.add(self.player)
        
        self.friend = Friend()
        self.friend_group.add(self.friend)
        
        #enemies
        self.enemy_1 = Enemy()
        self.enemy_group.add(self.enemy_1)
        self.enemy_2 = Enemy()
        self.enemy_group.add(self.enemy_2)
        self.enemy_3 = Enemy()
        self.enemy_group.add(self.enemy_3)
        self.enemy_4 = Enemy()
        self.enemy_group.add(self.enemy_4)        

        
        self.reward = 0
        self.total_reward = 0
        self.done = False
        self.agent = DQLAgent(weights)
        
    def findDistance(self,a, b):
        d = a-b
        return d
    
    def step(self,action):
        state_list = []
        
        #update
        self.player.update(action)
        self.enemy_group.update()
        self.friend_group.update()
        
        # Get Cordinates
        next_player_states = self.player.getCordinates()
        next_enemy_1_states = self.enemy_1.getCordinates()
        next_enemy_2_states = self.enemy_2.getCordinates()
        next_enemy_3_states = self.enemy_3.getCordinates()
        next_enemy_4_states = self.enemy_4.getCordinates()
        next_friend_1_states = self.friend.getCordinates()
        
        #Find Distances
        state_list.append(self.findDistance(next_player_states[0],next_enemy_1_states[0])) #e1 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_1_states[1])) #e1 y
        state_list.append(self.findDistance(next_player_states[0],next_enemy_2_states[0])) #e2 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_2_states[1])) #e2 y
        state_list.append(self.findDistance(next_player_states[0],next_enemy_3_states[0])) #e3 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_3_states[1])) #e3 y
        state_list.append(self.findDistance(next_player_states[0],next_enemy_4_states[0])) #e4 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_4_states[1])) #e4 y
        state_list.append(self.findDistance(next_player_states[0],next_friend_1_states[0])) #friend x
        state_list.append(self.findDistance(next_player_states[1],next_friend_1_states[1])) #friend y
        
        return [state_list] #len(state_list) = 10
    
    def initialStates(self): #reset()
        self.player_group = pygame.sprite.Group()
        self.enemy_group = pygame.sprite.Group()
        self.friend_group = pygame.sprite.Group()
        
        self.friend = Friend()
        self.friend_group.add(self.friend)
        
        self.player = Player()
        self.player_group.add(self.player)
        #enemies
        self.enemy_1 = Enemy()
        self.enemy_group.add(self.enemy_1)
        self.enemy_2 = Enemy()
        self.enemy_group.add(self.enemy_2)
        self.enemy_3 = Enemy()
        self.enemy_group.add(self.enemy_3)
        self.enemy_4 = Enemy()
        self.enemy_group.add(self.enemy_4)        
        
        self.reward = 0
        self.total_reward = 0
        self.done = False
        
        #state
        state_list = []
        # Get Cordinates
        player_states = self.player.getCordinates()
        enemy_1_states = self.enemy_1.getCordinates()
        enemy_2_states = self.enemy_2.getCordinates()
        enemy_3_states = self.enemy_3.getCordinates()
        enemy_4_states = self.enemy_4.getCordinates()
        friend_1_states = self.friend.getCordinates()
        
        
        #Find Distances
        state_list.append(self.findDistance(player_states[0],enemy_1_states[0])) #e1
        state_list.append(self.findDistance(player_states[1],enemy_1_states[1])) #e1
        state_list.append(self.findDistance(player_states[0],enemy_2_states[0])) #e2
        state_list.append(self.findDistance(player_states[1],enemy_2_states[1])) #e2
        state_list.append(self.findDistance(player_states[0],enemy_3_states[0])) #e3
        state_list.append(self.findDistance(player_states[1],enemy_3_states[1])) #e3
        state_list.append(self.findDistance(player_states[0],enemy_4_states[0])) #e4
        state_list.append(self.findDistance(player_states[1],enemy_4_states[1])) #e4
        state_list.append(self.findDistance(player_states[0],friend_1_states[0])) #friend x
        state_list.append(self.findDistance(player_states[1],friend_1_states[1])) #friend y

        
        return [state_list] #state
    
    def run(self):
        # Game Loop
        state = self.initialStates()
        batch_size = 32
        running = True
        while running:
            self.reward = 1
            # keep loop running at the right speed
            clock.tick(FPS)
            
            # process input
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    env.agent.target_model.save_weights("weights.h5")
                    testing = False
                    return

            #update
            action = self.agent.act(state)
            next_state = self.step(action)
            self.total_reward += self.reward

            # check to see if a enemy hit the player
            hits = pygame.sprite.spritecollide(self.player, self.enemy_group, False)
            if hits: #hits == True
                self.agent.targetModelUpdate()
                self.reward = -150
                self.total_reward += self.reward
                self.done = True
                running = False
                print("Total Reward: ", self.total_reward)
                
            hits_2 = pygame.sprite.spritecollide(self.player, self.friend_group, False)
            if hits_2: #hits == True
                self.reward = +30
                self.y_center = random.randint(10,HEIGHT-10)
                self.x_center = random.randint(WIDTH-265,WIDTH-10)
                self.friend.rect.center = (self.x_center,self.y_center)
                
            
            # remember (storage)
            self.agent.remember(state,action,self.reward, next_state, self.done)
            
            # update state
            state = next_state
            
            # training
            self.agent.replay(batch_size)
            
            # epsilon greedy
            self.agent.adaptiveEGreedy()                       
                
            #draw / render(show)
            screen.fill(BLACK)
            self.player_group.draw(screen)
            self.enemy_group.draw(screen)
            self.friend_group.draw(screen)
            
            #after drawing flip display
            pygame.display.flip()


                    
        pygame.quit()



if __name__ == '__main__':
    
    #Read weights that lastly been saved from educated model. 
    if "weights.h5" in os.listdir():
        weights = "weights.h5"
    else:
        weights= None
    #
    
    env = Env(weights)
    liste = []
    t = 0
    testing = True
    while testing:
        t += 1
        
        #Printing to the console
        print("Episode:",t)
        W = env.agent.target_model.get_weights()
        # print('weights:\n {}'.format(W))
        
        #Writing weights to csv
        file = open('WperEp.csv','a')
        file.write('Episode: {}\n Beginning Weights: {}\n'.format(t,W))
        file.close()
        
        liste.append(env.total_reward)

        # initialize pygame and create window
        pygame.init()
        screen = pygame.display.set_mode((WIDTH,HEIGHT))
        pygame.display.set_caption("kacma oyunu")
        clock = pygame.time.Clock()
        #Run Game
        env.run()
        
        file = open('WperEp.csv','a')
        file.write('Episode: {}\n Ending Weights: {}\n'.format(t,W))
        file.close()
        
        if not (t % EPISODENUM):
            env.agent.target_model.save_weights("weights.h5")
            testing = False







pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


Loaded model from disk
Loaded model from disk
Episode: 1
Total Reward:  -51
Episode: 2
Total Reward:  -42
Episode: 3
Total Reward:  105
Episode: 4
Total Reward:  200
Episode: 5
Total Reward:  -2
Episode: 6
Total Reward:  52
Episode: 7
Total Reward:  358
Episode: 8
Total Reward:  29
Episode: 9
Total Reward:  61
Episode: 10
Total Reward:  -39
Episode: 11
Total Reward:  165
Episode: 12
Total Reward:  9
Episode: 13
Total Reward:  -119
Episode: 14
Total Reward:  -122
Episode: 15
Total Reward:  6
Episode: 16
Total Reward:  604
Episode: 17
Total Reward:  -14
Episode: 18
Total Reward:  1608
Episode: 19
Total Reward:  184
Episode: 20
Total Reward:  911
Episode: 21
Total Reward:  -81
Episode: 22
Total Reward:  1770
Episode: 23
Total Reward:  249
Episode: 24
Total Reward:  288
Episode: 25
Total Reward:  -71
Episode: 26
Total Reward:  1229
Episode: 27
Total Reward:  -8
Episode: 28
Total Reward:  998
Episode: 29
Total Reward:  915
Episode: 30
Total Reward:  1415
Episode: 31
Total Reward:  158
Episo

Episode: 260
Total Reward:  81
Episode: 261
Total Reward:  -110
Episode: 262
Total Reward:  -77
Episode: 263
Total Reward:  344
Episode: 264
Total Reward:  -27
Episode: 265
Total Reward:  541
Episode: 266
Total Reward:  1244
Episode: 267
Total Reward:  -35
Episode: 268
Total Reward:  -117
Episode: 269
Total Reward:  815
Episode: 270
Total Reward:  715
Episode: 271
Total Reward:  1166
Episode: 272
Total Reward:  1966
Episode: 273
Total Reward:  -7
Episode: 274
Total Reward:  348
Episode: 275
Total Reward:  693
Episode: 276
Total Reward:  33
Episode: 277
Total Reward:  -77
Episode: 278
Total Reward:  138
Episode: 279
Total Reward:  93
Episode: 280
Total Reward:  382
Episode: 281
Total Reward:  318
Episode: 282
Total Reward:  686
Episode: 283
Total Reward:  400
Episode: 284
Total Reward:  250
Episode: 285
Total Reward:  896
Episode: 286
Total Reward:  280
Episode: 287
Total Reward:  156
Episode: 288
Total Reward:  -66
Episode: 289
Total Reward:  -100
Episode: 290
Total Reward:  -118
Episo

Total Reward:  -85
Episode: 517
Total Reward:  -120
Episode: 518
Total Reward:  948
Episode: 519
Total Reward:  88
Episode: 520
Total Reward:  0
Episode: 521
Total Reward:  -113
Episode: 522
Total Reward:  314
Episode: 523
Total Reward:  -10
Episode: 524
Total Reward:  31
Episode: 525
Total Reward:  -72
Episode: 526
Total Reward:  -17
Episode: 527
Total Reward:  -59
Episode: 528
Total Reward:  76
Episode: 529
Total Reward:  760
Episode: 530
Total Reward:  -67
Episode: 531
Total Reward:  92
Episode: 532
Total Reward:  -77
Episode: 533
Total Reward:  78
Episode: 534
Total Reward:  270
Episode: 535
Total Reward:  -117
Episode: 536
Total Reward:  119
Episode: 537
Total Reward:  -78
Episode: 538
Total Reward:  -32
Episode: 539
Total Reward:  648
Episode: 540
Total Reward:  -42
Episode: 541
Total Reward:  138
Episode: 542
Total Reward:  560
Episode: 543
Total Reward:  429
Episode: 544
Total Reward:  831
Episode: 545
Total Reward:  697
Episode: 546
Total Reward:  235
Episode: 547
Total Reward

...

Total Reward:  15
Episode: 4764
Total Reward:  -39
Episode: 4765
Total Reward:  528
Episode: 4766
Total Reward:  194
Episode: 4767
Total Reward:  320
Episode: 4768
Total Reward:  124
Episode: 4769
Total Reward:  110
Episode: 4770
Total Reward:  50
Episode: 4771
Total Reward:  127
Episode: 4772
Total Reward:  2
Episode: 4773
Total Reward:  15
Episode: 4774
Total Reward:  8
Episode: 4775
Total Reward:  -79
Episode: 4776
Total Reward:  291
Episode: 4777
Total Reward:  116
Episode: 4778
Total Reward:  946
Episode: 4779
Total Reward:  73
Episode: 4780
Total Reward:  939
Episode: 4781
Total Reward:  235
Episode: 4782
Total Reward:  41
Episode: 4783
Total Reward:  -102
Episode: 4784
Total Reward:  409
Episode: 4785
Total Reward:  269
Episode: 4786
Total Reward:  362
Episode: 4787
Total Reward:  226
Episode: 4788
Total Reward:  88
Episode: 4789
Total Reward:  76
Episode: 4790
Total Reward:  55
Episode: 4791
Total Reward:  -89
Episode: 4792
Total Reward:  34
Episode: 4793
Total Reward:  15
Epis

Total Reward:  -116
Episode: 5013
Total Reward:  658
Episode: 5014
Total Reward:  -10
Episode: 5015
Total Reward:  -10
Episode: 5016
Total Reward:  92
Episode: 5017
Total Reward:  430
Episode: 5018
Total Reward:  -117
Episode: 5019
Total Reward:  -34
Episode: 5020
Total Reward:  142
Episode: 5021
Total Reward:  965
Episode: 5022
Total Reward:  47
Episode: 5023
Total Reward:  311
Episode: 5024
Total Reward:  110
Episode: 5025
Total Reward:  204
Episode: 5026
Total Reward:  63
Episode: 5027
Total Reward:  203
Episode: 5028
Total Reward:  3
Episode: 5029
Total Reward:  -39
Episode: 5030
Total Reward:  101
Episode: 5031
Total Reward:  45
Episode: 5032
Total Reward:  -10
Episode: 5033
Total Reward:  470
Episode: 5034
Total Reward:  636
Episode: 5035
Total Reward:  -80
Episode: 5036
Total Reward:  -7
Episode: 5037
Total Reward:  483
Episode: 5038
Total Reward:  1678
Episode: 5039
Total Reward:  194
Episode: 5040
Total Reward:  -17
Episode: 5041
Total Reward:  309
Episode: 5042
Total Reward: 

Total Reward:  154
Episode: 5262
Total Reward:  926
Episode: 5263
Total Reward:  310
Episode: 5264
Total Reward:  283
Episode: 5265
Total Reward:  69
Episode: 5266
Total Reward:  196
Episode: 5267
Total Reward:  165
Episode: 5268
Total Reward:  336
Episode: 5269
Total Reward:  -33
Episode: 5270
Total Reward:  38
Episode: 5271
Total Reward:  -126
Episode: 5272
Total Reward:  20
Episode: 5273
Total Reward:  724
Episode: 5274
Total Reward:  787
Episode: 5275
Total Reward:  -114
Episode: 5276
Total Reward:  227
Episode: 5277
Total Reward:  -121
Episode: 5278
Total Reward:  1138
Episode: 5279
Total Reward:  357
Episode: 5280
Total Reward:  -106
Episode: 5281
Total Reward:  266
Episode: 5282
Total Reward:  -125
Episode: 5283
Total Reward:  512
Episode: 5284
Total Reward:  1104
Episode: 5285
Total Reward:  113
Episode: 5286
Total Reward:  478
Episode: 5287
Total Reward:  23
Episode: 5288
Total Reward:  -55
Episode: 5289
Total Reward:  85
Episode: 5290
Total Reward:  1490
Episode: 5291
Total R

Episode: 5510
Total Reward:  431
Episode: 5511
Total Reward:  351
Episode: 5512
Total Reward:  575
Episode: 5513
Total Reward:  -71
Episode: 5514
Total Reward:  -28
Episode: 5515
Total Reward:  332
Episode: 5516
Total Reward:  849
Episode: 5517
Total Reward:  96
Episode: 5518
Total Reward:  343
Episode: 5519
Total Reward:  669
Episode: 5520
Total Reward:  -102
Episode: 5521
Total Reward:  -114
Episode: 5522
Total Reward:  1969
Episode: 5523
Total Reward:  -123
Episode: 5524
Total Reward:  273
Episode: 5525
Total Reward:  409
Episode: 5526
Total Reward:  -33
Episode: 5527
Total Reward:  598
Episode: 5528
Total Reward:  344
Episode: 5529
Total Reward:  -15
Episode: 5530
Total Reward:  747
Episode: 5531
Total Reward:  73
Episode: 5532
Total Reward:  557
Episode: 5533
Total Reward:  288
Episode: 5534
Total Reward:  764
Episode: 5535
Total Reward:  202
Episode: 5536
Total Reward:  672
Episode: 5537
Total Reward:  327
Episode: 5538
Total Reward:  159
Episode: 5539
Total Reward:  -7
Episode: 

Total Reward:  -22
Episode: 5759
Total Reward:  -72
Episode: 5760
Total Reward:  42
Episode: 5761
Total Reward:  178
Episode: 5762
Total Reward:  119
Episode: 5763
Total Reward:  387
Episode: 5764
Total Reward:  1353
Episode: 5765
Total Reward:  -40
Episode: 5766
Total Reward:  -63
Episode: 5767
Total Reward:  70
Episode: 5768
Total Reward:  1117
Episode: 5769
Total Reward:  -124
Episode: 5770
Total Reward:  458
Episode: 5771
Total Reward:  53
Episode: 5772
Total Reward:  1104
Episode: 5773
Total Reward:  -26
Episode: 5774
Total Reward:  25
Episode: 5775
Total Reward:  50
Episode: 5776
Total Reward:  -27
Episode: 5777
Total Reward:  681
Episode: 5778
Total Reward:  859
Episode: 5779
Total Reward:  -59
Episode: 5780
Total Reward:  286
Episode: 5781
Total Reward:  -124
Episode: 5782
Total Reward:  677
Episode: 5783
Total Reward:  53
Episode: 5784
Total Reward:  506
Episode: 5785
Total Reward:  105
Episode: 5786
Total Reward:  23
Episode: 5787
Total Reward:  538
Episode: 5788
Total Reward